In [1]:
using MAT
using PCHIPInterpolation
include("juliaIC.jl");


function loadmat(matfile)
    file = matopen(matfile)
    con = read(file, "ElSpecOut") # note that this does NOT introduce a variable ``varname`` into scope
    close(file)
    return con
end

function getparams(con)
    ne = con["ne"] #size: (62, 674)
    
    iri = permutedims(con["iri"], (2, 1, 3))
    Tn_, Ti_, Te_, nN2, nO2, nO, nAr, nNOp, nO2p, nOp = [iri[i, :, :] for i in 1:size(iri)[1]]
    Tn = Tn_
    nNOp, nO2p, nOp = [i .* ne ./ sum([nNOp, nO2p, nOp]) for i in [nNOp, nO2p, nOp]]
    nion = cat(ne, nN2, nO2, nO, nAr, nNOp, nO2p, nOp; dims = 3)
    nion = permutedims(nion, (3, 1, 2))

    par = con["par"]
    Ti = par[:, 2, :]
    Te = par[:, 3, :]
    
    T = permutedims(cat(Te, Ti, Tn; dims = 3), (2, 1, 3)) #be aware of ordering!!!
    #size: (674, 62, 3)
    
    ts_ = dropdims(con["ts"]; dims = 2)
    te_ = dropdims(con["te"]; dims = 2)
    ts = ts_ .- ts_[1]
    te = te_ .- ts_[1]
    
    e_prod = permutedims(con["q"], (2, 1))
    #size 674x62
    
    h = dropdims(con["h"]; dims = 1)
    
    return ts, te, h, nion, T, e_prod
end


getparams (generic function with 1 method)

In [2]:
function interpolate_T(ts, T)
    i, j, k = size(T)
    T_intp = Array{Any}(undef, (j, k))

    for kk in 1:k
        for jj in 1:j
            T_intp[jj, kk] = Interpolator(ts, T[:, j, k])
        end
    end

    return T_intp
end

#function interpolate_q(ts, q)
#    i, k = size(q)
#    q_intp = Array{Any}(undef, k)
#
#    for kk in 1:k
#        q_intp[kk] = Interpolator(ts, q[:, k])
#    end
#
#    return T_intp
#end

interpolate_T (generic function with 1 method)

In [3]:
const path_reactions_file = "/Users/ost051/Documents/PhD/ElectronPrecipitation/Data/other/Reaction rates full set ext.txt"
const dndt, particles, reactions, ode_raw = juliaIC.initIC(path_reactions_file)
particles



19-element Vector{Vector{Any}}:
 [1, "O2+"]
 [2, "e-"]
 [3, "O(1D)"]
 [4, "O(1S)"]
 [5, "O"]
 [6, "N2+"]
 [7, "N(2D)"]
 [8, "N(4S)"]
 [9, "NO+"]
 [10, "O2+(a4P)"]
 [11, "O+(2D)"]
 [12, "O+(4S)"]
 [13, "O+(2P)"]
 [14, "O2"]
 [15, "NO"]
 [16, "N2"]
 [17, "N+"]
 [18, "H"]
 [19, "H+"]

In [13]:
function zerof(t)
    return 0
end

#make ion production part of ODE
function ion_production(qe, particles, nion)
    prod = Array{Any}(undef, length(particles))
    prod.= zerof
    i_O  = findall(p -> p[2] == "O", particles)[1]
    i_O2 = findall(p -> p[2] == "O2", particles)[1]
    i_N2 = findall(p -> p[2] == "N2", particles)[1]
    for i in 1:length(particles)
        if particles[i][2] == "e-"
            prod[i] = (t) -> qe(t, itp)
        end
        if particles[i][2] == "O+(4S)"
            prod[i] = (t) -> qe(t) .* n0[i_O , :]*0.56 ./(n0[i_O, :]*0.56 .+ n0[i_N2, :]*0.92 .+ n0[i_O2, :])
            println(size(prod[i](0)))
        end
        if particles[i][2] == "O2+"
            prod[i] = (t) -> qe(t) .* n0[i_N2, :]*0.92 ./(n0[i_O, :]*0.56 .+ n0[i_N2, :]*0.92 .+ n0[i_O2, :])
        end
        if particles[i][2] == "N2+"
            prod[i] = (t) -> qe(t) .* n0[i_O2, :]      ./(n0[i_O, :]*0.56 .+ n0[i_N2, :]*0.92 .+ n0[i_O2, :])
        end
    end

    return Array{Any}(undef, length(particles))
end


ion_production (generic function with 1 method)

In [14]:
function dummyf2(n, p, t)
    dn = zeros(size(n))
    reactions, nprod, dndt, T_intp, ih = p
    @views TT = [t(0) for t in T_intp[ih, :]]
    
    for j in 1:size(n)
        dn[j] = dndt[j](nprod, reactions, TT, n, t)[ih]
    end 
    return dn
end

function IC_byHeight()
    path_reactions_file = "/Users/ost051/Documents/PhD/ElectronPrecipitation/Data/other/Reaction rates full set ext.txt"
    dndt, particles, reactions, ode_raw = juliaIC.initIC(path_reactions_file)

    con = loadmat("/Users/ost051/Documents/PhD/ElectronPrecipitation/log/testing/2023.05.12_09_56_23_mixf=0/ElSpec-iqt_IC_0.mat")
    ts, te, h, nion, T, e_prod = getparams(con)
    n0 = nion[:, :, 1]
    nh = length(h)
    
    T_intp = interpolate_T(ts, T)
    println(size(T_intp))
    TT = Array{Float64}(undef, 3)
    
    #e_p_intp = interpolate_q(ts, e_prod)
    #println(size(e_p_intp))
    
    println(size(T))
    t_save = ts
    sol = Array{Any}(undef, nh)

    
    for ih in 1:nh
        #@time @views TT = [t(0) for t in T_intp[ih, :]]
        e_p_intp2 = Interpolator(ts, e_prod[:, ih])
        nprod = ion_production(e_p_intp2, particles, nion)
        #@time QQ = e_p_intp2(0)
        println("--")
        
        n0h = n0[:, ih]
        tspan = (ts, te)
        #prob = ODEProblem(dummyf2, n0h, tspan, (reactions, nprod, dndt, T_intp, ih))
        #sol[i] = solve(prob, TRBDF2(autodiff=false), reltol = 1e-7, abstol = 1e-3, saveat = t_save)
    end
    
    
    return 0
end

function setup()
end

setup()


IC_byHeight()

(62, 3)
(674, 62, 3)


LoadError: UndefVarError: `n0` not defined

In [6]:
con = loadmat("/Users/ost051/Documents/PhD/ElectronPrecipitation/log/testing/2023.05.12_09_56_23_mixf=0/ElSpec-iqt_IC_0.mat")
ts, te, h, ne, T, e_prod = getparams(con)
size(e_prod)


(674, 62)

In [7]:
"""
Performance:
1. Avoid containers with abstract types
    => define arrays as float64 explicitly


1. Type-stable code => no abstract types => define types explicitly as eg. Float64 of Int32
2. 

"""

"Performance:\n1. Avoid containers with abstract types\n    => define arrays as float64 explicitly\n\n\n1. Type-stable code => no abstract types => define types explicitly as eg. Float64 of Int32\n2. \n\n"